In [1]:
import os
import sys
import wave
from pocketsphinx import *

In [2]:
MODELDIR = 'C:\\Users\\pasm\\Documents\\Hack2020\\Silencio\\hack2020\\Lib\\site-packages\\pocketsphinx\\model'
BUFSIZE = 2048
THRESHOLD = 0

class AudioProcessor:
    def __init__(self):        
        self.config = Decoder.default_config()
        self.config.set_string('-hmm', os.path.join(MODELDIR, 'en-us'))
        self.config.set_string('-lm', os.path.join(MODELDIR, 'en-us.lm.bin'))
        self.config.set_string('-dict', os.path.join(MODELDIR, 'cmudict-en-us.dict'))
        self.config.set_string('-keyphrase', 'speak')
        self.config.set_float('-kws_threshold', 1e-5)
        self.config.set_float('-verbose', 0)
        self.decoder = Decoder(self.config)

    def ProcessAudioData(self, audioData):
        index = 0
        total = 0
        voice = 0
        self.decoder.start_utt()
        while (index < len(audioData)):
            total += 1
            chunk = audioData[index:(index+BUFSIZE)]
            self.decoder.process_raw(chunk, False, False)
            if self.decoder.hyp() != None:
                self.decoder.end_utt()
                voice += 1
                self.decoder.start_utt()
            index += BUFSIZE

        self.decoder.end_utt()
        return (voice/total > THRESHOLD)

In [3]:
inputFilePath = 'C:\\Users\\pasm\\Documents\\Hack2020\\Silencio\\Inputs\\SpeechWithMinNoise.wav'
sampleSizeInMs = 1000

audioProcessor = AudioProcessor()
aud = wave.open(inputFilePath, 'r')
ans = ''

framesPerSample = aud.getframerate() * sampleSizeInMs / 1000
counter = 0
muted = False
while (counter * framesPerSample < aud.getnframes()):
    counter += 1
    data = aud.readframes(int(framesPerSample))
    if audioProcessor.ProcessAudioData(data):
        ans += 'V'
        if (muted):
            muted = False
            print ('Unmute at ' + str(counter))
    else:
        ans += '.'
        if (not muted and ('V' not in ans[-3:])):
            muted = True
            print ('Mute at ' + str(counter))

aud.close()
print(ans)

Mute at 8
Unmute at 11
VVV.V.....VVVV
